# Dependence

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd

import numpy as np 

from pyDOE import lhs

import matplotlib.pyplot as plt
import matplotlib.tri as mtri

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameter

In [ ]:
steps = 10000
layers = np.array([2,32,64,32,2])
lr = 0.001

Nf = 2

# Auxiliary Functions


In [ ]:
def Plot(V,tris,U):
    _,ax=plt.subplots()
    ax.tripcolor(V[:,0],V[:,1],tris,U,cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('U')
    plt.show()

# Build initial physical mesh

In [ ]:
m_rows = 20
m_cols = 20

x = torch.linspace(0, 1, m_cols + 1)
y = torch.linspace(0, 1, m_rows + 1)
x = x.repeat(m_rows + 1)
y = y.repeat_interleave(m_cols + 1)
V = torch.stack([x,y],dim=1)

tris = []

for i in range(m_cols):
   for j in range(m_rows):
       tris.append([j * (m_cols + 1) + i, j * (m_cols + 1) + i + 1, (j + 1) * (m_cols + 1) + i + 1])
       tris.append([j * (m_cols + 1) + i, (j + 1) * (m_cols + 1) + i + 1, (j + 1) * (m_cols + 1) + i])
        
tris = torch.tensor(tris, dtype=torch.short)

left_boundarys = V[(V[:,0]==0.0)]
bottom_boundarys = V[(V[:,1]==0.0)]
top_boundarys = V[(V[:,1]==1.0)]
right_boundarys = V[(V[:,0]==1.0)]

plt.triplot(x,y,tris,'k-',linewidth=0.3)
plt.show()

# Training Data

In [ ]:
left_V = V[(V[:,0]==0.0)]
right_V = V[(V[:,0]==1.0)]
bottom_V = V[(V[:,1]==0.0)]
top_V = V[(V[:,1]==1.0)]

PDE_V = V[0] + (V[-1]-V[0])*lhs(2,Nf)

V = V.to(device)
left_V = left_V.float().to(device)
right_V = right_V.float().to(device)
bottom_V = bottom_V.float().to(device)
top_V = top_V.float().to(device)
PDE_V = PDE_V.float().to(device)

# Functions

In [ ]:
def U(V):
    return torch.tanh(-30*(V[:,1]-0.5-0.25*torch.sin(2*torch.pi*V[:,0])))

def U_x_y(V):
    U_x = (1-U(V)**2)*(15*torch.pi*torch.cos(2*torch.pi*V[:,0]))
    U_y = (1-U(V)**2)*(-30)
    return torch.stack([U_x,U_y],dim=1)

def Metric(V):
    u_x_y = U_x_y(V)
    return torch.sqrt(0.1*torch.linalg.norm(u_x_y,dim=1)**2 + 1)

# M = Metric(V)
# Plot(V.detach().cpu(),tris.detach().cpu(),M.detach().cpu())

# Neural Network generates initial logical mesh

$$
\begin{cases}
-\nabla \cdot (\nabla \xi) = 0, \\
\xi|_{\partial \Omega} = \xi_b,
\end{cases}
$$


In [ ]:
class FCN1(nn.Module):
    def __init__(self,layers):
        super().__init__()
        self.layers = layers 
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        self.loss_function2 = nn.L1Loss(reduction ='sum')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)   

    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        a = x.float()
        for i in range(len(self.layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)
        a = self.linears[-1](a)
        return a    
    
    def loss_BC(self,left_V,right_V,bottom_V,top_V):
        return self.loss_function2(self.forward(left_V),left_V) + \
               self.loss_function2(self.forward(right_V),right_V) + \
               self.loss_function2(self.forward(bottom_V),bottom_V) + \
               self.loss_function2(self.forward(top_V),top_V)
              
    def loss_PDE(self,PDE_V):
        g = PDE_V.clone()
        g.requires_grad = True
        xi_eta = self.forward(g)
        xi = xi_eta[:,0].unsqueeze(1)
        eta = xi_eta[:,1].unsqueeze(1)
        xi_x_y = autograd.grad(xi,g,torch.ones([g.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        xi_xx_yy = autograd.grad(xi_x_y,g,torch.ones(g.shape).to(device),create_graph=True)[0]
        eta_x_y = autograd.grad(eta,g,torch.ones([g.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        eta_xx_yy = autograd.grad(eta_x_y,g,torch.ones(g.shape).to(device),create_graph=True)[0]  
        return self.loss_function(xi_xx_yy[:,[0]]+xi_xx_yy[:,[1]],torch.zeros(g.shape[0],1).to(device)) + \
                self.loss_function(eta_xx_yy[:,[0]]+eta_xx_yy[:,[1]],torch.zeros(g.shape[0],1).to(device))

    def loss(self,left_V,right_V,bottom_V,top_V,PDE_V):      
        return self.loss_PDE(PDE_V) + self.loss_BC(left_V,right_V,bottom_V,top_V)

# Generate initial logic mesh

In [ ]:
xi_0 = FCN1(layers).to(device)
optimizer = torch.optim.Adam(xi_0.parameters(),lr=lr,amsgrad=False)

for i in range(steps):
    loss = xi_0.loss(left_V,right_V,bottom_V,top_V,PDE_V)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(i,loss.detach().cpu().numpy())
    
V_xi_0 = xi_0(V)
V_xi_0_cpu = xi_0(V).detach().cpu()
plt.triplot(V_xi_0_cpu[:,0],V_xi_0_cpu[:,1],tris.detach().cpu(),'k-',linewidth=0.3)


# Neural Network generate initial physical mesh by solving harmonic maps

$$
g^{ij}\frac{\partial}{\partial \xi^i}w\frac{\partial x^k}{\partial \xi^j} = 0
$$

$$
g^{ij} = \frac{\partial \xi^i}{\partial x^\alpha}\frac{\partial \xi^j}{\partial x^\alpha}
$$


In [ ]:
class FCN2(nn.Module):
    def __init__(self,layers):
        super().__init__()
        self.layers = layers 
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        self.loss_function2 = nn.L1Loss(reduction ='sum')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)   

    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        a = x.float()
        for i in range(len(self.layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)
        a = self.linears[-1](a)
        return a    
    
    def loss_BC(self,left_V,right_V,bottom_V,top_V):
        return self.loss_function2(self.forward(left_V),left_V) + \
               self.loss_function2(self.forward(right_V),right_V) + \
               self.loss_function2(self.forward(bottom_V),bottom_V) + \
               self.loss_function2(self.forward(top_V),top_V)
    
    def loss_PDE(self,PDE_V):
        g = PDE_V.clone()
        g.requires_grad = True
        xy = self.forward(g)
        x = xy[:,0].unsqueeze(1)
        y = xy[:,1].unsqueeze(1)
        M = Metric(xy).unsqueeze(1)
        x_xi_eta = autograd.grad(x,g,torch.ones([g.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        y_xi_eta = autograd.grad(y,g,torch.ones([g.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
        M_xi_eta = autograd.grad(M,g,torch.ones([g.shape[0],1]).to(device),create_graph=True)[0]
        x_xi = x_xi_eta[:,[0]]
        x_eta = x_xi_eta[:,[1]]
        y_xi = y_xi_eta[:,[0]]
        y_eta = y_xi_eta[:,[1]]
        x_xixi_xieta = autograd.grad(x_xi,g,torch.ones([g.shape[0],1]).to(device),create_graph=True)[0]
        x_etaxi_etaeta = autograd.grad(x_eta,g,torch.ones([g.shape[0],1]).to(device),create_graph=True)[0]
        y_xixi_xieta = autograd.grad(y_xi,g,torch.ones([g.shape[0],1]).to(device),create_graph=True)[0]
        y_etaxi_etaeta = autograd.grad(y_eta,g,torch.ones([g.shape[0],1]).to(device),create_graph=True)[0]
        g00 = (1/x_xi_eta[:,[0]])**2+(1/y_xi_eta[:,[0]])**2
        g01 = (1/x_xi_eta[:,[0]])*(1/x_xi_eta[:,[1]])+(1/y_xi_eta[:,[0]])*(1/y_xi_eta[:,[1]])
        g11 = (1/x_xi_eta[:,[1]])**2+(1/y_xi_eta[:,[1]])**2
        loss1 = g00*(M_xi_eta[:,[0]]*x_xi_eta[:,[0]]+M*x_xixi_xieta[:,[0]]) + \
                g01*(M_xi_eta[:,[0]]*x_xi_eta[:,[1]]+M*x_etaxi_etaeta[:,[0]]) + \
                g01*(M_xi_eta[:,[1]]*x_xi_eta[:,[0]]+M*x_xixi_xieta[:,[1]]) + \
                g11*(M_xi_eta[:,[1]]*x_xi_eta[:,[1]]+M*x_etaxi_etaeta[:,[1]])
        loss2 = g00*(M_xi_eta[:,[0]]*y_xi_eta[:,[0]]+M*y_xixi_xieta[:,[0]]) + \
                g01*(M_xi_eta[:,[0]]*y_xi_eta[:,[1]]+M*y_etaxi_etaeta[:,[0]]) + \
                g01*(M_xi_eta[:,[1]]*y_xi_eta[:,[0]]+M*y_xixi_xieta[:,[1]]) + \
                g11*(M_xi_eta[:,[1]]*y_xi_eta[:,[1]]+M*y_etaxi_etaeta[:,[1]])
        return self.loss_function(loss1,torch.zeros(g.shape[0],1).to(device)) + \
               self.loss_function(loss2,torch.zeros(g.shape[0],1).to(device))

    def loss(self,left_V,right_V,bottom_V,top_V,PDE_V):      
        return self.loss_PDE(PDE_V) + self.loss_BC(left_V,right_V,bottom_V,top_V)
    

# Generate initial physical mesh

In [ ]:
xi_new = FCN2(layers).to(device)
optimizer = torch.optim.Adam(xi_new.parameters(),lr=lr,amsgrad=False)

for i in range(1000):
    loss = xi_new.loss(left_V,right_V,bottom_V,top_V,PDE_V)
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(xi_new.parameters(), 1.0)
    optimizer.step()
    print(i,loss.detach().cpu().numpy())
    
V.requires_grad = True
V_xi_new = xi_new(V_xi_0)
V_xi_new_cpu = V_xi_new.detach().cpu()
plt.triplot(V_xi_new_cpu[:,0],V_xi_new_cpu[:,1],tris.detach().cpu(),'k-',linewidth=0.3)

# Update physical mesh

In [ ]:
xi_x_y = autograd.grad(V_xi_new[:,0].unsqueeze(1),V,torch.ones([V.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
eta_x_y = autograd.grad(V_xi_new[:,1].unsqueeze(1),V,torch.ones([V.shape[0],1]).to(device),retain_graph=True,create_graph=True)[0]
xi_x,xi_y,eta_x,eta_y = xi_x_y[:,[0]],xi_x_y[:,[1]],eta_x_y[:,[0]],eta_x_y[:,[1]]
delta_V_xi = V_xi_0 - V_xi_new
delta_V_xi_x = delta_V_xi[:,0].unsqueeze(1)
delta_V_xi_y = delta_V_xi[:,1].unsqueeze(1)
delta_V_x = (1/(xi_x*eta_y-xi_y*eta_x)*(eta_y*delta_V_xi_x-xi_y*delta_V_xi_y)).squeeze()
delta_V_y = (1/(xi_x*eta_y-xi_y*eta_x)*(-eta_x*delta_V_xi_x+xi_x*delta_V_xi_y)).squeeze()
delta_V = torch.stack([delta_V_x,delta_V_y],dim=1)
delta_V = delta_V / torch.linalg.norm(delta_V,dim=1).unsqueeze(1)*0.01
delta_V[V[:,0]==0.0] = 0.0
delta_V[V[:,0]==1.0] = 0.0
delta_V[V[:,1]==0.0] = 0.0
delta_V[V[:,1]==1.0] = 0.0

new_V = (V + delta_V).detach().cpu()

plt.triplot(new_V[:,0],new_V[:,1],tris.detach().cpu(),'k-',linewidth=0.3)
plt.show()